In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
dataFrames = pd.read_csv("/kaggle/input/cancer-data/Cancer_Data.csv")
print(dataFrames.keys())

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cancer-data/Cancer_Data.csv
Index(['id', 'diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean',
       'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean',
       'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
       'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',
       'fractal_dimension_se', 'radius_worst', 'texture_worst',
       'perimeter_worst', 'area_worst', 'smoothness_worst',
       'compactness_worst', 'concavity_worst', 'concave points_worst',
       'symmetry_worst', 'fractal_dimension_worst', 'Unnamed: 32'],
      dtype='object')


In [2]:
target = "diagnosis"
targetData = dataFrames[target]
inputData = dataFrames.drop([target, "id"],axis=1)

In [3]:
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split

labelEncoder = LabelEncoder()
targetData = labelEncoder.fit_transform(targetData)
X_train, X_test, y_train, y_test = train_test_split(
    inputData, targetData,
    train_size=0.8, 
    random_state=42, 
    stratify=targetData
)
pipe = Pipeline(steps=[
    ("Impule", SimpleImputer(strategy="mean")),
    ("transformer", StandardScaler())
])

X_train = pipe.fit_transform(X_train)
X_test = pipe.transform(X_test)



In [4]:
!pip install lazypredict


In [5]:
from lazypredict.Supervised import LazyClassifier

clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(X_train, X_test, y_train, y_test)
models

100%|██████████| 29/29 [00:02<00:00, 10.02it/s]


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
Perceptron,0.97,0.97,0.97,0.97,0.02
PassiveAggressiveClassifier,0.97,0.97,0.97,0.97,0.02
ExtraTreesClassifier,0.97,0.96,0.96,0.97,0.15
SVC,0.97,0.96,0.96,0.97,0.04
RandomForestClassifier,0.97,0.96,0.96,0.97,0.32
AdaBoostClassifier,0.96,0.96,0.96,0.96,0.20
LogisticRegression,0.96,0.96,0.96,0.96,0.05
LinearSVC,0.96,0.96,0.96,0.96,0.03
SGDClassifier,0.96,0.96,0.96,0.96,0.03


In [6]:
from sklearn.linear_model import Perceptron
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

clf = Perceptron()

params = {
    "penalty": ['l2','l1','elasticnet', None],
    "l1_ratio": [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1,0],
    "fit_intercept": [True, False],
    "max_iter": [100,200,300,400,500,600,700,800,900,1000,1100,1200,1300,1400,1500,1600],
    "tol": [1e-3,1e-4,1e-5,1e-6]
}
grid = GridSearchCV(clf, params, scoring="precision", cv=5, verbose=1, n_jobs=-1)
grid.fit(X_train, y_train)

y_predict = grid.predict(X_test)
print(grid.best_score_)
print(grid.best_params_)
print(classification_report(y_test, y_predict))

Fitting 5 folds for each of 5632 candidates, totalling 28160 fits
0.9783783783783784
{'fit_intercept': False, 'l1_ratio': 0.1, 'max_iter': 100, 'penalty': 'l2', 'tol': 0.001}
              precision    recall  f1-score   support

           0       0.97      0.97      0.97        72
           1       0.95      0.95      0.95        42

    accuracy                           0.96       114
   macro avg       0.96      0.96      0.96       114
weighted avg       0.96      0.96      0.96       114



In [7]:
model = Perceptron(fit_intercept=False, l1_ratio=0.1, penalty='l2', max_iter=100, tol=0.001)

model.fit(X_train, y_train)
model.score(X_test, y_test)

0.9649122807017544